# Catlab.jl is a package in Julia that implements Category Theory.

⊣ this symbol is "\vdash" 

⋅ this symbol is "\cdot"

→ this symbol is "\to"

⨟ this symbol is "\bbsemi"

In [ ]:
using Pkg
Pkg.activate(".")

## 1. First Steps - Recreating Category Theory
Below let's use Catlab in order to define a Category. This might seem odd, since Catlab is already Category Theory for Julia,
so why are we defining a Category? Well, first, this is already implemented, so we are doing it only to showcase. But also,
Catlab actually implement Generalized Algebraic Theories, which permits the definition of "things" other than Cateogories.
Thus, the package is actually more general.

In [1]:
using Pkg
Pkg.activate(".")
using Catlab

  Activating project at `~/Main/EMAp/Mathematical-Short-Notes/Fields/Category-Theory/notebooks`


In [2]:
@theory Category{Ob,Hom} begin
  @op begin
    (→) := Hom
    (⋅) := compose
  end

  Ob::TYPE
  Hom(dom::Ob, codom::Ob)::TYPE

  id(A::Ob)::(A → A)
  compose(f::(A → B), g::(B → C))::(A → C) ⊣ (A::Ob, B::Ob, C::Ob)
    

  (f ⋅ g) ⋅ h == f ⋅ (g ⋅ h) ⊣ (A::Ob, B::Ob, C::Ob, D::Ob,
                                f::(A → B), g::(B → C), h::(C → D))
  f ⋅ id(B) == f ⊣ (A::Ob, B::Ob, f::(A → B))
  id(A) ⋅ f == f ⊣ (A::Ob, B::Ob, f::(A → B))
end;

## 2 - Instantiating a Category


Now that we defined what a category is, let's instantiated it. But what does it mean? It means that we want to
use this idea of a category with existing Julia types. Let's do an easy example with
the category of Finite Vector Spaces. In this category,
morphisms are linear transformations and the objects are finite vector spaces
(which are all isomorphic to ℝⁿ for n the dimension).
Fortunantely, in Julia we have the type `Matrix{N,M}` and `MatrixDomain`. Thus, we can use `Matrix{N,M}` as the morphisms,
since matrices are linear transformations, and use `MatrixDomain` as our Finite Vector Space.

In [3]:
using LinearAlgebra: I

struct MatrixDomain
  eltype::Type
  dim::Int
end

@instance Category{MatrixDomain, Matrix} begin
  dom(M::Matrix) = MatrixDomain(eltype(M), size(M,1))
  codom(M::Matrix) = MatrixDomain(eltype(M), size(M,2))

  id(m::MatrixDomain) = Matrix{m.eltype}(I, m.dim, m.dim)
  compose(M::Matrix, N::Matrix) = M*N
end


A = rand(5,2)
B = rand(2,2)
@show dom(A), codom(A)
id(dom(A)) # The identity morphism is the identity matrix
# compose(B,A) <- not composable
compose(A,B)

(dom(A), codom(A)) = (MatrixDomain(Float64, 5), MatrixDomain(Float64, 2))


5×2 Matrix{Float64}:
 0.287896  0.171956
 0.634389  0.403799
 0.289773  0.350957
 0.426608  0.0983827
 0.387085  0.160465

Next, let's implement a more "convoluted" example. Let's use Catlab in order to create a
way to enforce that functions need to match dom/codom in order to be composed.
Note that does not happen naturally in Julia. A function is Julia does not have a parametric
type based on it's domain and codomain. Part of the reason is that most functions in Julia
do not have an output type enforcer (although this can be done, it's seldom used and 
actually desincentivised).

Hence, how can we guarantee that `compose(f,g)` actually can be composed, before evaluating the results?
The answer is with Catlab. But, before we show the example, here is 

In [14]:
struct Morphism{Input, Output}
    f::Function
end

(f::Morphism{Input, Output})(x::Input) where {Input, Output} = Output(f.f(x))

@instance Category{Type, Morphism} begin
  dom(f::Morphism) = typeof(f).parameters[1]
  codom(f::Morphism) = typeof(f).parameters[2]
  id(d::Type) = Morphism{d,d}(x->x)
  compose(g::Morphism, f::Morphism) = codom(f) === dom(g) ? Morphism{dom(f), codom(g)}(g ∘ f) : error("Domain and Codomain do not match.")
end

f = Morphism{Int, Int}(x->x)
g = Morphism{Int, Float64}(x->√x)

h = g ⋅ f
h(2)

1.4142135623730951

In [10]:
f ⋅ g

LoadError: Domain and Codomain do not match.

In [11]:
@show dom(f)
@show codom(f);

dom(f) = Int64
codom(f) = Int64


In [25]:
(g ⋅ id(Int))(10)

3.1622776601683795

## 2 - 

## 3 - Creating Functors

Let's now showcase the use of functors. Remember, a functor is like a morphism between categories, but it must preserve the
"domain/codomain" matching. I.e., if I have a category $\mathcal C$ and a category $\mathcal D$, my functor $F$ has to take each
object in $\mathcal C$ to an object in $\mathcal D$, and every morphism in $\mathcal C$ to a morphism in $\mathcal D$.

This might seem like a very convoluted thing, but the category of Preorders is a nice example. So, our 

In [ ]:
using Catlab.Theories:FreeThinCategory

In [ ]:
@theory Preorder{El,Leq} begin
  El::TYPE
  Leq(lhs::El, rhs::El)::TYPE
  @op (≤) := Leq

  # Preorder axioms are lifted to term constructors in the GAT.
  reflexive(A::El)::(A≤A) # ∀ A there is a term reflexive(A) which implies A≤A
  transitive(f::(A≤B), g::(B≤C))::(A≤C) ⊣ (A::El, B::El, C::El)

  # Axioms of the GAT are equivalences on terms or simplification rules in the logic
  f == g ⊣ (A::El, B::El, f::(A≤B), g::(A≤B))
  # Read as (f⟹ A≤B ∧ g⟹ A≤B) ⟹ f ≡ g
end

In [ ]:
typeof(FreeThinCategory.theory)

In [ ]:
@theory ThinCategory{Ob,Hom} <: Category{Ob,Hom} begin
  f == g ⊣ (A::Ob, B::Ob, f::Hom(A,B), g::Hom(A,B))
end

In [ ]:
@present P(FreeThinCategory) begin
  (X,Y,Z)::Ob
  f::Hom(X,Y)
  g::Hom(Y,Z)
end